In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from numba import jit

In [3]:
df = pd.read_csv('Preprocessed Fake Reviews Dataset.csv')

In [4]:
df.head()

,Unnamed: 0,category,rating,label,text_,length
0,0,Home_and_Kitchen_5,5.0,CG,love well made sturdi comfort love itveri pretti,48
1,1,Home_and_Kitchen_5,5.0,CG,love great upgrad origin ive mine coupl year,44
2,2,Home_and_Kitchen_5,5.0,CG,pillow save back love look feel pillow,38
3,3,Home_and_Kitchen_5,1.0,CG,miss inform use great product price,35
4,4,Home_and_Kitchen_5,5.0,CG,nice set good qualiti set two month,35


In [6]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
df.dropna(inplace=True)

In [14]:
df.groupby('label').describe()

rating                                                length  \
         count      mean       std  min  25%  50%  75%  max    count   
label                                                                  
CG     20215.0  4.259906  1.141092  1.0  4.0  5.0  5.0  5.0  20215.0   
OR     20216.0  4.253265  1.147652  1.0  4.0  5.0  5.0  5.0  20216.0   

                                                                
             mean         std  min   25%    50%    75%     max  
label                                                           
CG     154.067574  155.189232  4.0  51.0   88.0  192.0  1191.0  
OR     219.051247  233.256478  8.0  68.0  123.0  272.0  1868.0

In [9]:
def clean_text(text):
    nopunc = [w for w in text if w not in string.punctuation]
    nopunc = ''.join(nopunc)
    return  ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

In [10]:
def preprocess(text):
    return ' '.join([word for word in word_tokenize(text) if word not in stopwords.words('english') and not word.isdigit() and word not in string.punctuation])

In [11]:
stemmer = PorterStemmer()
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [12]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [15]:
review_train, review_test, label_train, label_test = train_test_split(df['text_'],df['label'],test_size=0.35)

In [16]:
def text_process(review):
    nopunc = [char for char in review if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [17]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',SVC())
])

In [18]:
pipeline.fit(review_train, label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x0000027A03D56820>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

In [19]:
pred = pipeline.predict(review_test)

In [20]:
from sklearn.metrics import accuracy_score
print('Accuracy Score:',accuracy_score(label_test,pred)*100)

Accuracy Score: 88.60857889901774


In [21]:
from joblib import parallel, delayed
import joblib

joblib.dump(pipeline, 'fake.pkl')

['fake.pkl']

In [22]:
txt = input()
txt = clean_text(txt)
txt = preprocess(txt)
txt = stem_words(txt)
txt = lemmatize_words(txt)
txt = [txt]
if pipeline.predict(txt)=='OR':
    print('Original')
else:
    print('Fake')

Original
